<a href="https://colab.research.google.com/github/msingh92-bit/mewp/blob/main/form5500_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Form 5500 Data Downloader

Downloads the following files from the DOL EBSA public database for **2009–2023**:

| Label | File | What it contains |
|---|---|---|
| `main_5500` | F_5500 | Plan identity, type, participants, plan year |
| `schedule_H` | F_SCH_H | Assets, liabilities, income, expenses, equity allocation |
| `schedule_R` | F_SCH_R | Retirement plan info, contribution rates |
| `schedule_R1` | F_SCH_R_PART1 | **Employer roster** — EINs of every contributing employer |

Files are saved to your **Google Drive** at `MyDrive/form5500/raw/{year}/`

**Run cells in order: 1 → 2 → 3 → 4 → 5 → 6**

---
**Estimated download size:** ~15–25 GB total across all years  
**Estimated time:** 45–90 minutes depending on your connection  
**Note:** Colab Pro recommended to avoid session timeouts on the full download.
For a quick test, run Cell 6 with `years=[2019]` first.

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print('Drive mounted.')

Mounted at /content/drive
Drive mounted.


In [3]:
# ── Cell 2: Imports and Configuration ───────────────────────────────────────

import os
import time
import zipfile
import csv
import requests
import pandas as pd
from pathlib import Path

# ── Where to save everything ──────────────────────────────────────────────
BASE_DIR = Path('/content/drive/MyDrive/form5500/raw')
BASE_DIR.mkdir(parents=True, exist_ok=True)
print(f'Base directory: {BASE_DIR}')

# ── Years to download ─────────────────────────────────────────────────────
# DOL switched to the current EFAST2 format in 2009.
# 2004-2008 have different URLs and file structures — handled separately below.
YEARS = list(range(2009, 2024))  # 2009 through 2023

# ── File types ────────────────────────────────────────────────────────────
# (label_for_your_reference, DOL_filename_stem)
FILE_TYPES = [
    ('main_5500',   'F_5500'),          # Main form: plan identity, type, participants
    ('schedule_H',  'F_SCH_H'),         # Assets, liabilities, equity allocation
    ('schedule_R',  'F_SCH_R'),         # Contribution rates, plan type
    ('schedule_R1', 'F_SCH_R_PART1'),   # EMPLOYER ROSTER — the linchpin file
]

# ── DOL URL base ──────────────────────────────────────────────────────────
# All files follow this pattern:
# https://askebsa.dol.gov/FOIA Files/{year}/Latest/{stem}_{year}_Latest.zip
DOL_BASE = 'https://askebsa.dol.gov/FOIA%20Files'

# ── Retry / politeness settings ───────────────────────────────────────────
MAX_RETRIES    = 3     # retry failed downloads this many times
RETRY_DELAY    = 15   # seconds to wait between retries
PAUSE_BETWEEN  = 3    # seconds to pause between each successful download

print(f'Will download {len(YEARS)} years × {len(FILE_TYPES)} file types = {len(YEARS)*len(FILE_TYPES)} zip files')
print(f'Years: {YEARS[0]}–{YEARS[-1]}')
print(f'File types: {[f[0] for f in FILE_TYPES]}')

Base directory: /content/drive/MyDrive/form5500/raw
Will download 15 years × 4 file types = 60 zip files
Years: 2009–2023
File types: ['main_5500', 'schedule_H', 'schedule_R', 'schedule_R1']


In [4]:
# ── Cell 3: Helper Functions ─────────────────────────────────────────────────

def build_url(year, file_stem):
    """Build the DOL download URL for a given year and file stem."""
    filename = f'{file_stem}_{year}_Latest.zip'
    return f'{DOL_BASE}/{year}/Latest/{filename}'


def download_file(url, dest_path, label):
    """
    Download url to dest_path with retry logic.
    Skips if the file already exists and is > 1KB (avoids re-downloading).
    Returns True on success, False on failure.
    """
    dest_path = Path(dest_path)
    dest_path.parent.mkdir(parents=True, exist_ok=True)

    # Skip if already downloaded and non-trivial size
    if dest_path.exists() and dest_path.stat().st_size > 1024:
        size_mb = dest_path.stat().st_size / (1024 * 1024)
        print(f'  [SKIP]  {label} already exists ({size_mb:.1f} MB)')
        return True

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            print(f'  [GET {attempt}/{MAX_RETRIES}] {label}')
            response = requests.get(url, timeout=180, stream=True)

            if response.status_code == 404:
                print(f'  [404]   {label} — file not found at DOL server')
                print(f'          URL: {url}')
                return False

            response.raise_for_status()  # raises on 4xx/5xx

            # Stream to disk in 1 MB chunks
            bytes_written = 0
            with open(dest_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024 * 1024):
                    f.write(chunk)
                    bytes_written += len(chunk)

            size_mb = bytes_written / (1024 * 1024)
            print(f'  [OK]    {label} — {size_mb:.1f} MB')
            return True

        except requests.exceptions.Timeout:
            print(f'  [TIMEOUT] {label} attempt {attempt} timed out')
        except requests.exceptions.RequestException as e:
            print(f'  [ERR]   {label} attempt {attempt}: {e}')

        if attempt < MAX_RETRIES:
            print(f'          Retrying in {RETRY_DELAY}s...')
            time.sleep(RETRY_DELAY)
        else:
            print(f'  [FAIL]  {label} — giving up after {MAX_RETRIES} attempts')
            return False


def unzip_file(zip_path, extract_dir):
    """
    Unzip zip_path into extract_dir.
    Skips if CSVs already present. Returns True on success.
    """
    zip_path    = Path(zip_path)
    extract_dir = Path(extract_dir)
    extract_dir.mkdir(parents=True, exist_ok=True)

    # Skip if already unzipped
    existing = list(extract_dir.glob('*.csv'))
    if existing:
        print(f'  [SKIP]  Already unzipped ({len(existing)} CSV files in {extract_dir.name}/)')
        return True

    try:
        with zipfile.ZipFile(zip_path, 'r') as z:
            z.extractall(extract_dir)
        csvs = list(extract_dir.glob('*.csv'))
        print(f'  [UNZIP] Extracted {len(csvs)} file(s) to {extract_dir.name}/')
        return True
    except zipfile.BadZipFile:
        print(f'  [ERR]   Bad zip — {zip_path.name} is corrupt (deleting for re-download)')
        zip_path.unlink(missing_ok=True)
        return False


def count_rows(csv_dir):
    """Count rows in the first CSV found in csv_dir. Returns 0 if none."""
    csvs = list(Path(csv_dir).glob('*.csv'))
    if not csvs:
        return 0
    with open(csvs[0], 'r', encoding='latin-1', errors='replace') as f:
        return sum(1 for _ in f) - 1  # subtract header


print('Helper functions defined.')

Helper functions defined.


In [5]:
# ── Cell 4: Main Download Function ───────────────────────────────────────────

def run_downloads(years=YEARS, file_types=FILE_TYPES, unzip=True, validate=True):
    """
    Download, unzip, and validate all Form 5500 files.
    Returns a list of result dicts; also writes download_manifest.csv to BASE_DIR.
    """
    print('=' * 65)
    print('Form 5500 Downloader')
    print(f'Years: {years[0]}–{years[-1]}  |  Files per year: {len(file_types)}')
    print(f'Save to: {BASE_DIR}')
    print('=' * 65)

    results = []

    for year in years:
        year_dir = BASE_DIR / str(year)
        year_dir.mkdir(parents=True, exist_ok=True)

        print(f'\n{"-"*55}')
        print(f'  YEAR {year}')
        print(f'{"-"*55}')

        for label, stem in file_types:
            url      = build_url(year, stem)
            zip_path = year_dir / f'{stem}_{year}_Latest.zip'
            csv_dir  = year_dir / stem

            # Step 1: Download
            dl_ok = download_file(url, zip_path, f'{year} / {label}')

            row_count = 0
            status    = 'DOWNLOAD_FAIL'

            if dl_ok:
                if unzip:
                    # Step 2: Unzip
                    uz_ok = unzip_file(zip_path, csv_dir)
                    if uz_ok:
                        if validate:
                            # Step 3: Count rows
                            row_count = count_rows(csv_dir)
                            if row_count >= 100:
                                status = 'OK'
                                print(f'  [VAL]   {row_count:,} rows — OK')
                            else:
                                status = 'WARN_LOW_ROWS'
                                print(f'  [WARN]  Only {row_count} rows — check file')
                        else:
                            status = 'UNZIPPED'
                    else:
                        status = 'UNZIP_FAIL'
                else:
                    status = 'DOWNLOADED'

            results.append({
                'year':      year,
                'file_type': label,
                'stem':      stem,
                'url':       url,
                'row_count': row_count,
                'status':    status,
            })

            time.sleep(PAUSE_BETWEEN)

    # Write manifest
    manifest_path = BASE_DIR / 'download_manifest.csv'
    with open(manifest_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=results[0].keys())
        writer.writeheader()
        writer.writerows(results)

    # Summary
    ok   = sum(1 for r in results if r['status'] == 'OK')
    skip = sum(1 for r in results if 'SKIP' in r['status'])
    fail = sum(1 for r in results if 'FAIL' in r['status'])
    warn = sum(1 for r in results if 'WARN' in r['status'])

    print(f'\n{"="*65}')
    print(f'COMPLETE — manifest saved to {manifest_path}')
    print(f'  ✓  OK:       {ok}')
    print(f'  ↷  Skipped:  {skip}  (already existed)')
    print(f'  ⚠  Warnings: {warn}  (low row count — inspect)')
    print(f'  ✗  Failed:   {fail}  (see manifest for URLs)')
    print(f'{"="*65}')

    return results


print('run_downloads() defined.')

run_downloads() defined.


In [6]:
# ── Cell 5: TEST FIRST — Download one year only ───────────────────────────────
# Run this before the full download to confirm everything works.
# 2019 is a good test year: post-GFC, pre-COVID, clean data.

test_results = run_downloads(
    years=[2019],
    file_types=FILE_TYPES
)

# If all 4 files show OK, proceed to Cell 6 for the full download.

Form 5500 Downloader
Years: 2019–2019  |  Files per year: 4
Save to: /content/drive/MyDrive/form5500/raw

-------------------------------------------------------
  YEAR 2019
-------------------------------------------------------
  [GET 1/3] 2019 / main_5500
  [OK]    2019 / main_5500 — 29.7 MB
  [UNZIP] Extracted 1 file(s) to F_5500/
  [VAL]   248,008 rows — OK
  [GET 1/3] 2019 / schedule_H
  [OK]    2019 / schedule_H — 15.2 MB
  [UNZIP] Extracted 1 file(s) to F_SCH_H/
  [VAL]   97,872 rows — OK
  [GET 1/3] 2019 / schedule_R
  [OK]    2019 / schedule_R — 2.4 MB
  [UNZIP] Extracted 1 file(s) to F_SCH_R/
  [VAL]   115,023 rows — OK
  [GET 1/3] 2019 / schedule_R1
  [OK]    2019 / schedule_R1 — 0.2 MB
  [UNZIP] Extracted 1 file(s) to F_SCH_R_PART1/
  [VAL]   7,148 rows — OK

COMPLETE — manifest saved to /content/drive/MyDrive/form5500/raw/download_manifest.csv
  ✓  OK:       4
  ↷  Skipped:  0  (already existed)
  ⚠  Warnings: 0  (low row count — inspect)
  ✗  Failed:   0  (see manifest f

In [7]:
# ── Cell 6: FULL DOWNLOAD — All years 2009-2023 ───────────────────────────────
# Only run after Cell 5 confirms the test year works.
# This will take 45-90 minutes. Already-downloaded files are skipped.
# Safe to re-run if interrupted — it picks up where it left off.

results = run_downloads(
    years=YEARS,
    file_types=FILE_TYPES
)

Form 5500 Downloader
Years: 2009–2023  |  Files per year: 4
Save to: /content/drive/MyDrive/form5500/raw

-------------------------------------------------------
  YEAR 2009
-------------------------------------------------------
  [GET 1/3] 2009 / main_5500
  [OK]    2009 / main_5500 — 49.1 MB
  [UNZIP] Extracted 1 file(s) to F_5500/
  [VAL]   410,919 rows — OK
  [GET 1/3] 2009 / schedule_H
  [OK]    2009 / schedule_H — 14.5 MB
  [UNZIP] Extracted 1 file(s) to F_SCH_H/
  [VAL]   96,067 rows — OK
  [GET 1/3] 2009 / schedule_R
  [OK]    2009 / schedule_R — 3.9 MB
  [UNZIP] Extracted 1 file(s) to F_SCH_R/
  [VAL]   186,113 rows — OK
  [GET 1/3] 2009 / schedule_R1
  [OK]    2009 / schedule_R1 — 0.2 MB
  [UNZIP] Extracted 1 file(s) to F_SCH_R_PART1/
  [VAL]   10,184 rows — OK

-------------------------------------------------------
  YEAR 2010
-------------------------------------------------------
  [GET 1/3] 2010 / main_5500
  [OK]    2010 / main_5500 — 39.0 MB
  [UNZIP] Extracted 1 file

In [8]:
# ── Cell 7: Review Results ────────────────────────────────────────────────────

manifest = pd.read_csv(BASE_DIR / 'download_manifest.csv')

print('Row counts by year and file type:')
print('(These are record counts in the CSV — gives a sense of scale)')
print()
pivot = manifest.pivot_table(
    index='year',
    columns='file_type',
    values='row_count',
    aggfunc='sum'
)
print(pivot.to_string())

# Anything that needs attention:
problems = manifest[~manifest['status'].isin(['OK', 'SKIP'])]
if len(problems) > 0:
    print(f'\n⚠  {len(problems)} files need attention:')
    print(problems[['year', 'file_type', 'status', 'url']].to_string())
else:
    print('\n✓ All files OK.')

Row counts by year and file type:
(These are record counts in the CSV — gives a sense of scale)

file_type  main_5500  schedule_H  schedule_R  schedule_R1
year                                                     
2009          410919       96067      186113        10184
2010          322879       94207      153267         8106
2011          291337       92184      134779         6137
2012          277046       92040      127073         8935
2013          270682       92090      123507        12273
2014          268226       93016      121295        11471
2015          262544       93044      121311        10825
2016          256816       94112      117894         9622
2017          251960       94913      116055         7756
2018          246545       96025      114131         7403
2019          248008       97872      115023         7148
2020          249548      100009      114026         6835
2021          244072       99510      111980         6541
2022          244024      100641 

In [9]:
# ── Cell 8: Sanity Check — Peek at Files ────────────────────────────────────
# Open a sample year of each file type and print shape + column names.
# This confirms the CSVs are readable and have the expected structure.

SAMPLE_YEAR = 2019

for label, stem in FILE_TYPES:
    csv_dir = BASE_DIR / str(SAMPLE_YEAR) / stem
    csvs = list(csv_dir.glob('*.csv'))
    if not csvs:
        print(f'\n{stem}: NO CSV FOUND in {csv_dir}')
        continue

    df = pd.read_csv(csvs[0], nrows=3, encoding='latin-1', low_memory=False)
    print(f'\n{"-"*55}')
    print(f'{label} ({stem}) — {csvs[0].name}')
    print(f'  Columns: {len(df.columns)}')
    print(f'  First 5 columns: {list(df.columns[:5])}')
    # Key columns to confirm present:
    key_cols = {
        'main_5500':   ['ACK_ID', 'PLAN_NAME', 'SPONS_DFE_EIN', 'PLAN_NUM'],
        'schedule_H':  ['ACK_ID', 'SCH_H_TOT_ASSETS_BOY_AMT', 'SCH_H_TOT_ASSETS_EOY_AMT'],
        'schedule_R':  ['ACK_ID', 'SCH_R_TOT_PARTCP_BOY_CNT'],
        'schedule_R1': ['ACK_ID', 'SCH_R_CONTRIBING_EMPL_EIN'],
    }
    if label in key_cols:
        found    = [c for c in key_cols[label] if c in df.columns]
        missing  = [c for c in key_cols[label] if c not in df.columns]
        print(f'  Key columns present: {found}')
        if missing:
            print(f'  ⚠ Missing expected columns: {missing}')
            print(f'    (column names may differ by year — check layout file)')

print(f'\n{"-"*55}')
print('Sanity check complete.')
print(f'Files are at: {BASE_DIR}')


-------------------------------------------------------
main_5500 (F_5500) — f_5500_2019_latest.csv
  Columns: 135
  First 5 columns: ['ACK_ID', 'FORM_PLAN_YEAR_BEGIN_DATE', 'FORM_TAX_PRD', 'TYPE_PLAN_ENTITY_CD', 'TYPE_DFE_PLAN_ENTITY_CD']
  Key columns present: ['ACK_ID', 'PLAN_NAME', 'SPONS_DFE_EIN']
  ⚠ Missing expected columns: ['PLAN_NUM']
    (column names may differ by year — check layout file)

-------------------------------------------------------
schedule_H (F_SCH_H) — F_SCH_H_2019_latest.csv
  Columns: 155
  First 5 columns: ['ACK_ID', 'SCH_H_PLAN_YEAR_BEGIN_DATE', 'SCH_H_TAX_PRD', 'SCH_H_PN', 'SCH_H_EIN']
  Key columns present: ['ACK_ID']
  ⚠ Missing expected columns: ['SCH_H_TOT_ASSETS_BOY_AMT', 'SCH_H_TOT_ASSETS_EOY_AMT']
    (column names may differ by year — check layout file)

-------------------------------------------------------
schedule_R (F_SCH_R) — F_SCH_R_2019_latest.csv
  Columns: 60
  First 5 columns: ['ACK_ID', 'SCH_R_PLAN_YEAR_BEGIN_DATE', 'SCH_R_TAX_PRD',

In [16]:
# ── Cell 9: Early Years Download — 1999 to 2008 (EFAST1 format) ────────────
#
# Source: https://www.dol.gov/agencies/ebsa/about-ebsa/our-activities/
#         public-disclosure/foia/form-5500-datasets
#
# ── What changed vs. the original Cell 9 ────────────────────────────────────
#   OLD: covered 2004-2008 only, wrong URL pattern
#   NEW: covers 1999-2008 (earliest publicly available without a FOIA request)
#        Uses correct askebsa.dol.gov URLs confirmed from DOL source page
#
# ── Key structural differences from 2009+ ────────────────────────────────────
#
#   FILING SYSTEM : EFAST1 (not EFAST2). Different column names, different
#                   field structure. You will need to harmonize columns when
#                   building a combined 1999-2023 panel.
#
#   URL PATTERN   : No Latest/ or All/ subdirectory for pre-2009 years.
#                   Pattern: askebsa.dol.gov/FOIA Files/{year}/{stem}_{year}.zip
#
#   SCHEDULE R    : DID NOT EXIST before 2009. Schedule R (employer roster,
#                   contribution rates, multiemployer info) was introduced with
#                   EFAST2 and the 2009 form redesign. There is NO equivalent
#                   file for 1999-2008. This is a hard limit — the data simply
#                   does not exist in structured form. For your paper, the
#                   employer-plan linkage is only available from 2009 onward.
#
#   WHAT YOU GET  :
#     ✓  F_5500      — Main form: plan EIN, plan type, participant counts
#     ✓  F_SCH_H     — Schedule H: assets, liabilities, benefit expenses,
#                      investment income — everything you need for plan-level
#                      financial analysis and pre-shock equity allocation
#     ✗  F_SCH_R     — NOT available (did not exist pre-2009)
#     ✗  F_SCH_R_PART1 — NOT available (did not exist pre-2009)
#
#   PRIOR TO 1999 : Data before 1999 requires a formal FOIA request to DOL.
#                   Contact: ERISA Public Disclosure Room, (202) 693-8673
#                   Source: DOL page states explicitly "For datasets prior to
#                   1999... submit a request by following the instructions on
#                   our FOIA page. There may be a fee."
#
# ── Research use for 1999-2008 ───────────────────────────────────────────────
#   Even without Schedule R, these years give you:
#   1. Plan-level asset/liability data for pre-period characterization
#   2. Equity allocations going into the 2001-02 dot-com shock (key instrument)
#   3. Benefit expense trends to validate pre-trends assumption
#   4. Plan participant counts for size controls
# ─────────────────────────────────────────────────────────────────────────────

EARLY_YEARS = list(range(1999, 2009))   # 1999 through 2008 — full range available

# Only F_5500 and F_SCH_H — Schedule R does not exist for these years
EARLY_FILE_TYPES = [
    ('main_5500',  'F_5500'),    # Main form ✓
    ('schedule_H', 'F_SCH_H'),  # Schedule H financials ✓
    # schedule_R and schedule_R1 intentionally excluded — not available pre-2009
]

# Correct base URL for early years (no Latest/ subdirectory)
EARLY_DOL_BASE = 'https://askebsa.dol.gov/FOIA%20Files'


def build_early_url(year, file_stem):
    """
    Build the correct DOL URL for 1999-2008 files.
    These use a flat structure with no Latest/ or All/ subdirectory.
    Example: https://askebsa.dol.gov/FOIA Files/2005/F_5500_2005.zip
    """
    return f'{EARLY_DOL_BASE}/{year}/{file_stem}_{year}.zip'


def build_early_layout_url(year, file_stem):
    """Layout file URL (column definitions) for early years."""
    return f'{EARLY_DOL_BASE}/{year}/{file_stem}_{year}_layout.txt'


def run_early_downloads(years=EARLY_YEARS, file_types=EARLY_FILE_TYPES):
    """
    Download, unzip, and validate Form 5500 files for 1999-2008.
    Schedule R intentionally skipped — does not exist for these years.
    Returns a list of result dicts; writes download_manifest_1999_2008.csv.
    """
    print('=' * 65)
    print('Form 5500 Early Years Downloader  |  1999-2008  |  EFAST1')
    print(f'Years: {years[0]}-{years[-1]}  |  Files per year: {len(file_types)}')
    print(f'Files: {[f[0] for f in file_types]}')
    print(f'Note: Schedule R/R_PART1 not available pre-2009 — skipped')
    print(f'Save to: {BASE_DIR}')
    print('=' * 65)

    results = []

    for year in years:
        year_dir = BASE_DIR / str(year)
        year_dir.mkdir(parents=True, exist_ok=True)

        print(f'\n{"-"*55}')
        print(f'  YEAR {year}  (EFAST1 format)')
        print(f'{"-"*55}')

        for label, stem in file_types:
            url      = build_early_url(year, stem)
            zip_path = year_dir / f'{stem}_{year}.zip'
            csv_dir  = year_dir / stem

            # Try to grab the layout file too (column definitions — useful for
            # harmonizing with 2009+ EFAST2 column names later)
            layout_url  = build_early_layout_url(year, stem)
            layout_path = year_dir / f'{stem}_{year}_layout.txt'
            if not layout_path.exists():
                try:
                    r = requests.get(layout_url, timeout=20)
                    if r.status_code == 200:
                        layout_path.write_bytes(r.content)
                        print(f'  [LAYOUT] Saved {layout_path.name}')
                except Exception:
                    pass   # layout files are optional; don't fail on missing

            # Download the zip
            dl_ok = download_file(url, zip_path, f'{year} / {label}')

            row_count = 0
            status    = 'DOWNLOAD_FAIL'

            if dl_ok:
                uz_ok = unzip_file(zip_path, csv_dir)
                if uz_ok:
                    row_count = count_rows(csv_dir)
                    if row_count >= 100:
                        status = 'OK'
                        print(f'  [VAL]   {row_count:,} rows — OK')
                    else:
                        status = 'WARN_LOW_ROWS'
                        print(f'  [WARN]  Only {row_count} rows — inspect file')
                else:
                    status = 'UNZIP_FAIL'

            results.append({
                'year':          year,
                'file_type':     label,
                'efast_version': 'EFAST1',
                'schedule_R':    'NOT_AVAILABLE_PRE_2009',
                'url':           url,
                'row_count':     row_count,
                'status':        status,
            })

            time.sleep(PAUSE_BETWEEN)

    # Write manifest for early years
    manifest_path = BASE_DIR / 'download_manifest_1999_2008.csv'
    if results:
        with open(manifest_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=results[0].keys())
            writer.writeheader()
            writer.writerows(results)

    # Print summary
    ok   = sum(1 for r in results if r['status'] == 'OK')
    skip = sum(1 for r in results if 'SKIP' in r['status'])
    warn = sum(1 for r in results if 'WARN' in r['status'])
    fail = sum(1 for r in results if 'FAIL' in r['status'])

    print(f'\n{"="*65}')
    print(f'EARLY YEARS COMPLETE  ({years[0]}-{years[-1]})')
    print(f'  ✓  OK:       {ok}')
    print(f'  ↷  Skipped:  {skip}  (already existed)')
    print(f'  ⚠  Warnings: {warn}')
    print(f'  ✗  Failed:   {fail}')
    print(f'  Manifest: {manifest_path}')
    print()
    print('IMPORTANT — column harmonization needed before combining with 2009+ data:')
    print('  EFAST1 (1999-2008) uses different column names than EFAST2 (2009+).')
    print('  The layout .txt files downloaded above define the column structure.')
    print('  Key fields to harmonize: plan EIN, net assets, benefit expenses,')
    print('  equity allocation share. Check layout files side-by-side.')
    print(f'{"="*65}')

    return results


# ── Test with one year first ─────────────────────────────────────────────────
# Uncomment one line below and run:
# early_results = run_early_downloads(years=[2005])   # single year test

# ── Full early years download (1999-2008) ────────────────────────────────────
early_results = run_early_downloads()


Form 5500 Early Years Downloader  |  1999-2008  |  EFAST1
Years: 1999-2008  |  Files per year: 2
Files: ['main_5500', 'schedule_H']
Note: Schedule R/R_PART1 not available pre-2009 — skipped
Save to: /content/drive/MyDrive/form5500/raw

-------------------------------------------------------
  YEAR 1999  (EFAST1 format)
-------------------------------------------------------
  [LAYOUT] Saved F_5500_1999_layout.txt
  [GET 1/3] 1999 / main_5500
  [OK]    1999 / main_5500 — 76.1 MB
  [UNZIP] Extracted 0 file(s) to F_5500/
  [WARN]  Only 0 rows — inspect file
  [LAYOUT] Saved F_SCH_H_1999_layout.txt
  [GET 1/3] 1999 / schedule_H
  [OK]    1999 / schedule_H — 10.3 MB
  [UNZIP] Extracted 0 file(s) to F_SCH_H/
  [WARN]  Only 0 rows — inspect file

-------------------------------------------------------
  YEAR 2000  (EFAST1 format)
-------------------------------------------------------
  [LAYOUT] Saved F_5500_2000_layout.txt
  [GET 1/3] 2000 / main_5500
  [OK]    2000 / main_5500 — 108.8 MB
  

In [17]:
# ── Cell 10: Download Data Dictionaries ──────────────────────────────────────
#
# Data dictionaries define every field in every file — column names, types,
# descriptions, valid codes. Essential before you write any cleaning code.
#
# ── Coverage by era ──────────────────────────────────────────────────────────
#
#   2009-2024  EFAST2  → Full data dictionary zip per year
#              URL: dol.gov/sites/dolgov/files/EBSA/.../form-5500-{year}-data-dictionary.zip
#              Contains: Excel workbook with one tab per schedule
#              (F_5500, F_SCH_H, F_SCH_R, F_SCH_R_PART1, etc.)
#
#   1999-2008  EFAST1  → No data dictionary zip. Column definitions are in
#              the _layout.txt files downloaded alongside each zip in Cell 9.
#              Those files are already on your Drive if you ran Cell 9.
#
# ── What to do with these ────────────────────────────────────────────────────
#   1. Open the Excel dictionary for the year you're working on
#   2. Find the tab for the schedule you care about (e.g. F_SCH_H)
#   3. Cross-reference column names when writing your Stata/Python cleaning code
#   4. When harmonizing 1999-2008 with 2009+ data, compare layout .txt files
#      against the 2009 data dictionary — that's where column names changed
# ─────────────────────────────────────────────────────────────────────────────

import os, time, requests, csv
from pathlib import Path

# ── Configuration ─────────────────────────────────────────────────────────────
# BASE_DIR already defined in Cell 2. If running this cell standalone, set it:
# BASE_DIR = Path('/content/drive/MyDrive/form5500/raw')

DICT_DIR = BASE_DIR / 'data_dictionaries'
DICT_DIR.mkdir(parents=True, exist_ok=True)

# Data dictionary base URL (dol.gov, not askebsa.dol.gov)
DICT_BASE = 'https://www.dol.gov/sites/dolgov/files/EBSA/about-ebsa/our-activities/public-disclosure/foia'

# ── Which years to download for ───────────────────────────────────────────────
# Change this to whatever year(s) you need. Default: all EFAST2 years.
DICT_YEARS = list(range(2009, 2025))   # 2009 through 2024

# For a single year only:
# DICT_YEARS = [2019]

MAX_RETRIES   = 3
RETRY_DELAY   = 10
PAUSE_BETWEEN = 2


def build_dict_url(year):
    """Build the data dictionary URL for a given year."""
    return f'{DICT_BASE}/form-5500-{year}-data-dictionary.zip'


def download_dictionary(year):
    """
    Download and unzip the data dictionary for a given year.
    Saves to BASE_DIR/data_dictionaries/{year}/
    Returns a result dict.
    """
    url      = build_dict_url(year)
    zip_path = DICT_DIR / f'form-5500-{year}-data-dictionary.zip'
    ext_dir  = DICT_DIR / str(year)
    ext_dir.mkdir(exist_ok=True)

    # Skip if already downloaded
    if zip_path.exists() and zip_path.stat().st_size > 1024:
        size_kb = zip_path.stat().st_size / 1024
        print(f'  [SKIP]  {year} — already downloaded ({size_kb:.0f} KB)')
        return {'year': year, 'url': url, 'status': 'SKIP', 'note': ''}

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            print(f'  [GET {attempt}/{MAX_RETRIES}] {year} data dictionary')
            r = requests.get(url, timeout=60, stream=True)

            if r.status_code == 404:
                # Not all years have a dictionary zip — fall back gracefully
                print(f'  [404]   {year} — no data dictionary zip at DOL')
                print(f'          For {year}, use the layout .txt files instead.')
                return {'year': year, 'url': url, 'status': 'NOT_FOUND',
                        'note': 'No dictionary zip — use layout .txt files'}

            r.raise_for_status()

            with open(zip_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=512 * 1024):
                    f.write(chunk)

            size_kb = zip_path.stat().st_size / 1024
            print(f'  [OK]    {year} — {size_kb:.0f} KB')

            # Unzip
            import zipfile
            try:
                with zipfile.ZipFile(zip_path, 'r') as z:
                    z.extractall(ext_dir)
                files = list(ext_dir.iterdir())
                print(f'  [UNZIP] {len(files)} file(s) extracted to data_dictionaries/{year}/')
                note = ', '.join(f.name for f in files)
            except zipfile.BadZipFile:
                print(f'  [WARN]  {year} zip is corrupt — deleting for re-download')
                zip_path.unlink(missing_ok=True)
                note = 'BadZipFile'

            return {'year': year, 'url': url, 'status': 'OK', 'note': note}

        except requests.exceptions.RequestException as e:
            print(f'  [ERR]   {year} attempt {attempt}: {e}')
            if attempt < MAX_RETRIES:
                time.sleep(RETRY_DELAY)

    return {'year': year, 'url': url, 'status': 'FAIL', 'note': 'all retries exhausted'}


# ── Run ───────────────────────────────────────────────────────────────────────
print('=' * 65)
print(f'Downloading data dictionaries for {len(DICT_YEARS)} years: {DICT_YEARS[0]}-{DICT_YEARS[-1]}')
print(f'Save to: {DICT_DIR}')
print('=' * 65)
print()
print('NOTE: 1999-2008 (EFAST1) have no dictionary zip.')
print('      Their column layouts were saved in Cell 9 as _layout.txt files.')
print()

dict_results = []

for year in DICT_YEARS:
    result = download_dictionary(year)
    dict_results.append(result)
    time.sleep(PAUSE_BETWEEN)

# ── Save manifest ─────────────────────────────────────────────────────────────
manifest_path = DICT_DIR / 'dictionary_manifest.csv'
with open(manifest_path, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=dict_results[0].keys())
    writer.writeheader()
    writer.writerows(dict_results)

# ── Summary ───────────────────────────────────────────────────────────────────
ok        = sum(1 for r in dict_results if r['status'] == 'OK')
skipped   = sum(1 for r in dict_results if r['status'] == 'SKIP')
not_found = sum(1 for r in dict_results if r['status'] == 'NOT_FOUND')
failed    = sum(1 for r in dict_results if r['status'] == 'FAIL')

print(f'\n{"="*65}')
print(f'DONE — {len(DICT_YEARS)} years processed')
print(f'  ✓  Downloaded:  {ok}')
print(f'  ↷  Skipped:    {skipped}  (already existed)')
print(f'  ○  Not found:  {not_found}  (no zip at DOL — use layout .txt)')
print(f'  ✗  Failed:     {failed}')
print(f'  Manifest: {manifest_path}')
print()
print(f'Files at: {DICT_DIR}')
print()
print('Each year folder contains an Excel workbook (.xlsx or .xls).')
print('Open it and look for these tabs:')
print('  F_5500          — main form fields')
print('  F_SCH_H         — Schedule H (assets, liabilities, equity allocation)')
print('  F_SCH_R         — Schedule R (contribution rates, plan type)')
print('  F_SCH_R_PART1   — Schedule R Part 5 (employer roster — your key file)')
print(f'{"="*65}')


Save to: /content/drive/MyDrive/form5500/raw/data_dictionaries

NOTE: 1999-2008 (EFAST1) have no dictionary zip.
      Their column layouts were saved in Cell 9 as _layout.txt files.

  [GET 1/3] 2009 data dictionary
  [OK]    2009 — 5915 KB
  [UNZIP] 13 file(s) extracted to data_dictionaries/2009/
  [GET 1/3] 2010 data dictionary
  [OK]    2010 — 7081 KB
  [UNZIP] 13 file(s) extracted to data_dictionaries/2010/
  [GET 1/3] 2011 data dictionary
  [OK]    2011 — 7034 KB
  [UNZIP] 12 file(s) extracted to data_dictionaries/2011/
  [GET 1/3] 2012 data dictionary
  [OK]    2012 — 3298 KB
  [UNZIP] 12 file(s) extracted to data_dictionaries/2012/
  [GET 1/3] 2013 data dictionary
  [OK]    2013 — 9675 KB
  [UNZIP] 12 file(s) extracted to data_dictionaries/2013/
  [GET 1/3] 2014 data dictionary
  [OK]    2014 — 4433 KB
  [UNZIP] 11 file(s) extracted to data_dictionaries/2014/
  [GET 1/3] 2015 data dictionary
  [OK]    2015 — 7665 KB
  [UNZIP] 1 file(s) extracted to data_dictionaries/2015/
  [GE

In [18]:
# ── Cell 11: Download 5500-SF, Schedule A, and Schedule A Part 1 ─────────────
#
# PURPOSE: Get pension contribution and health/welfare contribution data
#          for single-employer firms (and multiemployer plans where reported).
#
# ── Files downloaded ──────────────────────────────────────────────────────────
#
#   F_5500_SF         Short Form 5500-SF
#                     Filed by small plans (< 100 participants, unfunded/insured).
#                     Contains plan type, participant counts, funding arrangement.
#                     Key for identifying single-employer vs. multiemployer plans
#                     in the small-plan universe.
#
#   F_SCH_A           Schedule A — Insurance Information
#                     Reports insurance contracts held by the plan.
#                     KEY FIELD: premiums paid to insurers for health/welfare
#                     benefits. This is your direct measure of health & welfare
#                     contributions flowing through insured arrangements.
#                     Also covers pension annuity contracts.
#
#   F_SCH_A_PART1     Schedule A Part 1 — Insurance Carrier Detail
#                     One row per insurance contract per plan.
#                     Contains: carrier name, carrier EIN, contract type code,
#                     premiums paid, benefits paid, experience-rated amounts.
#                     Contract type codes distinguish pension vs. health/welfare:
#                       Code 1 = Health/Medical
#                       Code 2 = Life insurance
#                       Code 3 = Dental/Vision
#                       Code 4 = Temporary disability
#                       Code 5 = Long-term disability
#                       Code 6 = Long-term care
#                       Code 7 = Pension/annuity
#                     Use Part 1 to split pension vs. H&W contributions cleanly.
#
# ── Availability by era ───────────────────────────────────────────────────────
#
#   2009-2024  EFAST2  : All three files available with consistent structure.
#              URL pattern: askebsa.dol.gov/FOIA Files/{year}/Latest/{stem}_{year}_Latest.zip
#
#   1999-2008  EFAST1  : F_5500_SF and F_SCH_A available but different column
#              names. F_SCH_A_PART1 may not exist as a separate file pre-2009
#              (carrier detail was embedded differently in EFAST1).
#              URL pattern: askebsa.dol.gov/FOIA Files/{year}/{stem}_{year}.zip
#
# ── Research use ──────────────────────────────────────────────────────────────
#   Schedule A + A_PART1 give you the most granular view of H&W contributions
#   for insured single-employer plans. For self-funded (trust-based) plans,
#   contributions appear in Schedule H instead. Your full H&W contribution
#   measure = Schedule A premiums (insured) + Schedule H benefit expenses
#   (self-funded), identified by plan type code on the main F_5500 form.
#
# ── Files saved into same year folders as Cells 6 and 9 ─────────────────────
#   MyDrive/form5500/raw/{year}/F_5500_SF/
#   MyDrive/form5500/raw/{year}/F_SCH_A/
#   MyDrive/form5500/raw/{year}/F_SCH_A_PART1/
# ─────────────────────────────────────────────────────────────────────────────

import time, zipfile, csv, requests
from pathlib import Path

# BASE_DIR already defined in Cell 2.
# If running this cell standalone, uncomment and set:
# from google.colab import drive
# drive.mount('/content/drive')
# BASE_DIR = Path('/content/drive/MyDrive/form5500/raw')

# ── Files to download ─────────────────────────────────────────────────────────
# (label, stem, note)
SCH_A_FILES = [
    ('sf_5500',       'F_5500_SF',       'Short Form 5500-SF — small plan filings'),
    ('schedule_A',    'F_SCH_A',         'Schedule A — insurance premiums by plan'),
    ('schedule_A1',   'F_SCH_A_PART1',   'Schedule A Part 1 — carrier-level detail, pension vs. H&W split'),
]

# ── Year ranges ───────────────────────────────────────────────────────────────
EFAST2_YEARS = list(range(2009, 2024))   # 2009-2023: Latest/ subdirectory, _Latest suffix
EFAST1_YEARS = list(range(1999, 2009))   # 1999-2008: flat URL, no suffix

# To download a subset of years only, replace with e.g.:
# EFAST2_YEARS = [2018, 2019, 2020]
# EFAST1_YEARS = []

# ── URL builders ──────────────────────────────────────────────────────────────
DOL_BASE = 'https://askebsa.dol.gov/FOIA%20Files'

def build_url_efast2(year, stem):
    """2009+ URL: .../FOIA Files/{year}/Latest/{stem}_{year}_Latest.zip"""
    return f'{DOL_BASE}/{year}/Latest/{stem}_{year}_Latest.zip'

def build_url_efast1(year, stem):
    """1999-2008 URL: .../FOIA Files/{year}/{stem}_{year}.zip"""
    return f'{DOL_BASE}/{year}/{stem}_{year}.zip'

# ── Core helpers (re-used from Cell 3) ───────────────────────────────────────
# download_file(), unzip_file(), count_rows() must be defined (run Cell 3 first)

def download_and_extract(url, zip_path, csv_dir, label):
    """Download zip, unzip to csv_dir, count rows. Returns status dict."""
    zip_path = Path(zip_path)
    csv_dir  = Path(csv_dir)
    csv_dir.mkdir(parents=True, exist_ok=True)

    dl_ok = download_file(url, zip_path, label)
    if not dl_ok:
        return {'status': 'DOWNLOAD_FAIL', 'row_count': 0}

    uz_ok = unzip_file(zip_path, csv_dir)
    if not uz_ok:
        return {'status': 'UNZIP_FAIL', 'row_count': 0}

    row_count = count_rows(csv_dir)
    if row_count >= 100:
        print(f'  [VAL]   {row_count:,} rows — OK')
        return {'status': 'OK', 'row_count': row_count}
    else:
        print(f'  [WARN]  Only {row_count} rows — inspect file')
        return {'status': 'WARN_LOW_ROWS', 'row_count': row_count}


def run_sch_a_downloads():
    """
    Download F_5500_SF, F_SCH_A, and F_SCH_A_PART1 for all years.
    Saves into the same year folders used by Cells 6 and 9.
    """
    all_years    = [(y, 'EFAST2') for y in EFAST2_YEARS] + [(y, 'EFAST1') for y in EFAST1_YEARS]
    total_files  = len(all_years) * len(SCH_A_FILES)

    print('=' * 65)
    print('Schedule A / A Part 1 / 5500-SF Downloader')
    print(f'EFAST2 years (2009+): {EFAST2_YEARS[0] if EFAST2_YEARS else "none"}-{EFAST2_YEARS[-1] if EFAST2_YEARS else "none"}')
    print(f'EFAST1 years (pre-2009): {EFAST1_YEARS[0] if EFAST1_YEARS else "none"}-{EFAST1_YEARS[-1] if EFAST1_YEARS else "none"}')
    print(f'Files: {[f[0] for f in SCH_A_FILES]}')
    print(f'Total zips: {total_files}')
    print(f'Save to: {BASE_DIR}/{{year}}/{{file_type}}/')
    print('=' * 65)

    results = []

    for year, era in all_years:
        year_dir = BASE_DIR / str(year)
        year_dir.mkdir(parents=True, exist_ok=True)

        print(f'\n{"-"*55}')
        print(f'  YEAR {year}  ({era})')
        print(f'{"-"*55}')

        for label, stem, note in SCH_A_FILES:

            # F_SCH_A_PART1 may not exist as a separate file in EFAST1.
            # Try the URL anyway — the 404 handler will catch it gracefully.
            if era == 'EFAST1':
                url = build_url_efast1(year, stem)
            else:
                url = build_url_efast2(year, stem)

            zip_name = f'{stem}_{year}_Latest.zip' if era == 'EFAST2' else f'{stem}_{year}.zip'
            zip_path = year_dir / zip_name
            csv_dir  = year_dir / stem      # e.g. .../2019/F_SCH_A/

            outcome = download_and_extract(
                url      = url,
                zip_path = zip_path,
                csv_dir  = csv_dir,
                label    = f'{year} / {label}'
            )

            results.append({
                'year':       year,
                'era':        era,
                'file_type':  label,
                'stem':       stem,
                'note':       note,
                'url':        url,
                'csv_dir':    str(csv_dir),
                'row_count':  outcome['row_count'],
                'status':     outcome['status'],
            })

            time.sleep(PAUSE_BETWEEN)

    # ── Manifest ──────────────────────────────────────────────────────────────
    manifest_path = BASE_DIR / 'download_manifest_sch_a.csv'
    with open(manifest_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=results[0].keys())
        writer.writeheader()
        writer.writerows(results)

    # ── Summary ───────────────────────────────────────────────────────────────
    ok        = sum(1 for r in results if r['status'] == 'OK')
    skipped   = sum(1 for r in results if 'SKIP' in r['status'])
    not_found = sum(1 for r in results if r['status'] == 'NOT_FOUND')
    failed    = sum(1 for r in results if 'FAIL' in r['status'])
    warned    = sum(1 for r in results if 'WARN' in r['status'])

    print(f'\n{"="*65}')
    print(f'SCHEDULE A DOWNLOAD COMPLETE')
    print(f'  ✓  OK:         {ok}')
    print(f'  ↷  Skipped:   {skipped}  (already existed)')
    print(f'  ○  Not found: {not_found}  (not available for that year/era)')
    print(f'  ⚠  Warnings:  {warned}')
    print(f'  ✗  Failed:    {failed}')
    print(f'  Manifest: {manifest_path}')
    print()
    print('HOW TO USE THESE FILES:')
    print()
    print('  1. Identify single-employer plans:')
    print('     F_5500: TYPE_PLAN_ENTITY_CD = 1  (single employer)')
    print('     F_5500: TYPE_PLAN_ENTITY_CD = 2  (plan of controlled group)')
    print('     F_5500: COLLECTIVE_BARGAIN_IND = Y  (flags collectively bargained plans)')
    print()
    print('  2. Split pension vs. H&W contributions in F_SCH_A_PART1:')
    print('     INS_CONTRACT_TYPE_CD = 1  → Health/Medical (H&W)')
    print('     INS_CONTRACT_TYPE_CD = 2  → Life insurance (H&W)')
    print('     INS_CONTRACT_TYPE_CD = 3  → Dental/Vision (H&W)')
    print('     INS_CONTRACT_TYPE_CD = 4  → Temporary disability (H&W)')
    print('     INS_CONTRACT_TYPE_CD = 5  → Long-term disability (H&W)')
    print('     INS_CONTRACT_TYPE_CD = 6  → Long-term care (H&W)')
    print('     INS_CONTRACT_TYPE_CD = 7  → Pension/annuity contracts')
    print()
    print('  3. Key contribution amount fields in F_SCH_A and F_SCH_A_PART1:')
    print('     SCH_A_PREM_PAID_TOT_AMT    Total premiums paid')
    print('     SCH_A_PREM_DUE_EOY_AMT     Premiums due at year-end')
    print('     SCH_A_BENEF_PAID_TOT_AMT   Total benefits paid under contract')
    print()
    print('  4. For self-funded plans (no insurance contract), H&W contributions')
    print('     appear in F_SCH_H: SCH_H_BENEF_PAID_HLTH_WLFR_AMT')
    print('     Your total H&W measure = Schedule A premiums (insured plans)')
    print('                            + Schedule H benefit expenses (self-funded)')
    print(f'{"="*65}')

    return results


# ── Test with one year first ──────────────────────────────────────────────────
# Uncomment to test before running the full download:
# EFAST2_YEARS_BAK, EFAST1_YEARS_BAK = EFAST2_YEARS, EFAST1_YEARS
# EFAST2_YEARS, EFAST1_YEARS = [2019], []
# test = run_sch_a_downloads()
# EFAST2_YEARS, EFAST1_YEARS = EFAST2_YEARS_BAK, EFAST1_YEARS_BAK

# ── Full download ─────────────────────────────────────────────────────────────
sch_a_results = run_sch_a_downloads()


Schedule A / A Part 1 / 5500-SF Downloader
EFAST2 years (2009+): 2009-2023
EFAST1 years (pre-2009): 1999-2008
Files: ['sf_5500', 'schedule_A', 'schedule_A1']
Total zips: 75
Save to: /content/drive/MyDrive/form5500/raw/{year}/{file_type}/

-------------------------------------------------------
  YEAR 2009  (EFAST2)
-------------------------------------------------------
  [GET 1/3] 2009 / sf_5500
  [OK]    2009 / sf_5500 — 69.0 MB
  [UNZIP] Extracted 1 file(s) to F_5500_SF/
  [VAL]   453,560 rows — OK
  [GET 1/3] 2009 / schedule_A
  [OK]    2009 / schedule_A — 18.0 MB
  [UNZIP] Extracted 1 file(s) to F_SCH_A/
  [VAL]   321,422 rows — OK
  [GET 1/3] 2009 / schedule_A1
  [OK]    2009 / schedule_A1 — 11.6 MB
  [UNZIP] Extracted 1 file(s) to F_SCH_A_PART1/
  [VAL]   394,596 rows — OK

-------------------------------------------------------
  YEAR 2010  (EFAST2)
-------------------------------------------------------
  [GET 1/3] 2010 / sf_5500
  [OK]    2010 / sf_5500 — 78.1 MB
  [UNZIP] Ex